In [ ]:
import re
import pandas as pd
import numpy as np

# Paquetes llamadas 
import time

# Scrapping 
import requests

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### **Extracción de empresas**

In [2]:
data = []

for pa in range(1, 5):
    resp = requests.post("https://startupshub.catalonia.com/investments-in-catalan-startups?pageNumber=4&year=0", verify=False)
    soup = BeautifulSoup(resp.text, "html.parser")
    
    # Buscar todos los elementos con la clase 'items'
    for item in soup.find_all("div", class_="items"):
        # Extraer la URL desde el atributo `url` en la etiqueta <a>
        url_element = item.find("a", attrs={"url": True})
        url = url_element["url"] if url_element else None

        # Extraer información
        startup_name = item.find("h4").text.strip() if item.find("h4") else None
        company_name = item.find("div", class_="item-text").find_all("p")[0].text.strip()
        category = item.find("div", class_="item-text").find_all("p")[1].text.strip()
        investment_amount = item.find("div", class_="col-md-4").find("p").text.strip()
        
        investors_text = item.find("div", class_="col-md-4").find("strong")
        investors = investors_text.find_next_sibling(string=True).strip() if investors_text else None
        
        date = item.find("p", class_="date").text.strip() if item.find("p", class_="date") else None

        # Agregar datos a la lista
        data.append([startup_name, company_name, category, investment_amount, investors, date, url])

# Crear DataFrame
df = pd.DataFrame(data, columns=["Startup", "Company", "Category", "Investment", "Investors", "Date", "URL"])

d:\DOCENCIA\CURS\TFG Alumnos\2024-2025\S02\JORDI CAPDEVILA\syntax\entorno\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'startupshub.catalonia.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\DOCENCIA\CURS\TFG Alumnos\2024-2025\S02\JORDI CAPDEVILA\syntax\entorno\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'startupshub.catalonia.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\DOCENCIA\CURS\TFG Alumnos\2024-2025\S02\JORDI CAPDEVILA\syntax\entorno\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'startupshub.catalonia.com'. Adding certificate verificatio

### **Extracción de info de cada empresa**

In [4]:
lista = []

In [ ]:
for i in range(0, df.shape[0]):
    # Configurar el driver (asegúrate de poner la ruta correcta a tu WebDriver)
    driver = webdriver.Chrome()  # Usa webdriver.Firefox() si usas Firefox

    # Nos quedamos con la url 
    url = "https://startupshub.catalonia.com/" + df.loc[i, "URL"]

    # Extraemos la información de la URL
    resp = requests.get(url, verify=False)
    soup = BeautifulSoup(resp.text, "html.parser")

    # Extraer información relevante
    name = soup.find('h1', class_='big_title').text.strip()
    description = soup.find('meta', {'name': 'description'})['content']
    address = soup.find('strong', class_='company-name').find_next_sibling(string=True).strip()
    founded = soup.find('span', class_='founded').find('strong').text.strip()
    employees = soup.find('span', class_='employers').find('strong').text.strip()
    industries = soup.find('span', class_='industries information-item').text.replace('Industries:', '').strip()
    technologies = soup.find('span', class_='technologies information-item').text.replace('Technologies:', '').strip()
    others_fields = soup.find('span', class_='otherFields information-item').text.replace('Other fields:', '').strip()
    financial_founded = soup.find('span', class_='founded information-item').text.replace('Founded:', '').strip()
    business_model = soup.find('span', class_='model information-item').text.replace('Business model:', '').strip()
    targets = soup.find('span', class_='target information-item').text.replace('Target:', '').strip()
    spinoff = soup.find('span', class_='spinoffs information-item').text.replace('Spinoff participants:', '').strip()
    funding_info = []
    for funding in soup.find_all('div', class_='item punts-servei-content row-fluid'):
        amount = funding.find('h3').text.strip()
        date = funding.find('h3').text.strip().split('(')[-1].replace(')', '')
        # investors = funding.find('p', text=lambda t: t and 'Investors:' in t).text.replace('Investors:', '').strip()
        # funding_info.append({'amount': amount, 'date': date, 'investors': investors})
        funding_info.append({'amount': amount, 'date': date})

    # Cargar la página
    time.sleep(10)
    driver.get(url)
    
    financial_funding_stage = soup.find('span', class_='stage information-item').text.replace('Funding stage:', '').strip()
    financial_employees = soup.find('span', class_='employers information-item').text.replace('Employees:', '').strip()

    financial_funding_stage = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "fundingStageTxt"))).text
    financial_employees = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "employeesTxt"))).text

    founding = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "generic-container")))[0].text

    # Cerrar el navegador
    driver.quit()

    data = {
            'Name': [name],
            'Description': [description],
            'Address': [address],
            'Industries': [industries],
            'Technologies': [technologies],
            'Other fields': [others_fields],
            'Funding stage': [financial_funding_stage],
            'Founded': [financial_founded],
            'Employees': [financial_employees],
            'Business model': [business_model],
            'Target': [targets],
            'Spinoff participants': [spinoff],
            'Funding': [funding_info], 
            'Funding2': [founding]
        }
    
    lista.append(data)

d:\DOCENCIA\CURS\TFG Alumnos\2024-2025\S02\JORDI CAPDEVILA\syntax\entorno\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'startupshub.catalonia.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\DOCENCIA\CURS\TFG Alumnos\2024-2025\S02\JORDI CAPDEVILA\syntax\entorno\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'startupshub.catalonia.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\DOCENCIA\CURS\TFG Alumnos\2024-2025\S02\JORDI CAPDEVILA\syntax\entorno\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'startupshub.catalonia.com'. Adding certificate verificatio

KeyError: 76

In [47]:
lista

[{'Name': ['EQUITO APP S.L'],
  'Description': ['A real estate investment platform enabling users to invest in fractional properties starting from €100. The platform facilitates seamless investment management and offers passive monthly income through property tokenization and fractional ownership.'],
  'Address': ['Paseig De Gracia 19 08007 Barcelona'],
  'Industries': ['Proptech, \r\n\t\t                \t\t\r\n\t\t                \t\t\tFinancing'],
  'Technologies': ['Fintech & Insurtech, \r\n\t\t                \t\t\r\n\t\t                \t\t\tDLT/Blockchain'],
  'Other fields': ['Social Economy'],
  'Funding stage': ['Seed (0.25 M EUR - < 1 M EUR)'],
  'Founded': ['2021'],
  'Employees': ['From 11 to 20'],
  'Business model': ['Marketplace'],
  'Target': ['Consumer'],
  'Spinoff participants': ['None / Not a spinoff'],
  'Funding': [[{'amount': '0.3M € (June 2024)', 'date': 'June 2024'}]],
  'Funding2': ['Funding\n0.3M € (JUNE 2024)\nCapital\nInvestors: Lanzadera\nSource: Press re

In [51]:
df

,Startup,Company,Category,Investment,Investors,Date,URL
0,EQUITO,EQUITO APP S.L,FinancingProptech,0.3M €,Lanzadera,June 2024,startup/barcelona/equito/5843
1,VORA,VORA,Fashion & Design,0.3M €,Casilda Güell i Javier Sobrino,June 2024,startup/barcelona/vora/6706
2,LEXDOKA,SEEDFY ECOSYSTEM S.L.,Business Services & SoftwareLegaltech,0.3M €,Codols Finances,May 2024,startup/barcelona/lexdoka/6563
3,KOKUAI,KOKUAI S.L.,HealthtechBusiness Services & Software,0.3M €,"Elisabeth Martinez, Oriol Vila, Blast",September 2024,startup/barcelona/kokuai/5838
4,FLAMAID,FLAMAID S.L.,Hardware,0.3M €,ONA Capital,July 2024,startup/barcelona/flamaid/7137
...,...,...,...,...,...,...,...
71,COCOPOOL,PLACE4PLAN S.L.,E-commerce & MarketplacesTraveltech & Leisure,0.1M €,Fundadors,June 2024,startup/barcelona/cocopool/6433
72,OMASHU,OMASHU S.L.,HealthtechGamingLegaltechSports,0.1M €,"tres efes' (familiars, amics i entusiastes)",April 2024,startup/barcelona/omashu/5895
73,GET-PIZZA,APPASSIONE S.L.,FoodLogistic Tech,0.0M €,Fundadors,June 2024,startup/barcelona/get-pizza/6718
74,LAAGAM,CID BRAND SL,Fashion & Design,,Tomas Meyer,October 2024,startup/barcelona/laagam/5599


In [ ]:
df_info = pd.DataFrame(lista)
df_info["Name"] = [d[0] for d in df_info["Name"]]

In [61]:
df

,Startup,Company,Category,Investment,Investors,Date,URL
0,EQUITO,EQUITO APP S.L,FinancingProptech,0.3M €,Lanzadera,June 2024,startup/barcelona/equito/5843
1,VORA,VORA,Fashion & Design,0.3M €,Casilda Güell i Javier Sobrino,June 2024,startup/barcelona/vora/6706
2,LEXDOKA,SEEDFY ECOSYSTEM S.L.,Business Services & SoftwareLegaltech,0.3M €,Codols Finances,May 2024,startup/barcelona/lexdoka/6563
3,KOKUAI,KOKUAI S.L.,HealthtechBusiness Services & Software,0.3M €,"Elisabeth Martinez, Oriol Vila, Blast",September 2024,startup/barcelona/kokuai/5838
4,FLAMAID,FLAMAID S.L.,Hardware,0.3M €,ONA Capital,July 2024,startup/barcelona/flamaid/7137
...,...,...,...,...,...,...,...
71,COCOPOOL,PLACE4PLAN S.L.,E-commerce & MarketplacesTraveltech & Leisure,0.1M €,Fundadors,June 2024,startup/barcelona/cocopool/6433
72,OMASHU,OMASHU S.L.,HealthtechGamingLegaltechSports,0.1M €,"tres efes' (familiars, amics i entusiastes)",April 2024,startup/barcelona/omashu/5895
73,GET-PIZZA,APPASSIONE S.L.,FoodLogistic Tech,0.0M €,Fundadors,June 2024,startup/barcelona/get-pizza/6718
74,LAAGAM,CID BRAND SL,Fashion & Design,,Tomas Meyer,October 2024,startup/barcelona/laagam/5599


In [70]:
df_info = df_info.drop_duplicates(subset=['Name'], keep='first')

In [ ]:
datosFinal = df.merge(df_info, left_on='Company', right_on='Name')

,Startup,Company,Category,Investment,Investors,Date,URL,Name,Description,Address,...,Technologies,Other fields,Funding stage,Founded,Employees,Business model,Target,Spinoff participants,Funding,Funding2
0,EQUITO,EQUITO APP S.L,FinancingProptech,0.3M €,Lanzadera,June 2024,startup/barcelona/equito/5843,EQUITO APP S.L,[A real estate investment platform enabling us...,[Paseig De Gracia 19 08007 Barcelona],...,"[Fintech & Insurtech, \r\n\t\t ...",[Social Economy],[Seed (0.25 M EUR - < 1 M EUR)],[2021],[From 11 to 20],[Marketplace],[Consumer],[None / Not a spinoff],"[[{'amount': '0.3M € (June 2024)', 'date': 'Ju...",[Funding\n0.3M € (JUNE 2024)\nCapital\nInvesto...
1,VORA,VORA,Fashion & Design,0.3M €,Casilda Güell i Javier Sobrino,June 2024,startup/barcelona/vora/6706,VORA,"[Vora is inspired by the city of Barcelona, pr...","[Calle Cavallers, 54 P. 4 Pta. 1 08034 Barcelona]",...,[E-commerce],[Social Economy],[Seed (0.25 M EUR - < 1 M EUR)],[2022],[Undisclosed],[-],[Consumer],[None / Not a spinoff],"[[{'amount': '0.3M € (June 2024)', 'date': 'Ju...",[Funding\n0.3M € (JUNE 2024)\nCapital\nInvesto...
2,LEXDOKA,SEEDFY ECOSYSTEM S.L.,Business Services & SoftwareLegaltech,0.3M €,Codols Finances,May 2024,startup/barcelona/lexdoka/6563,SEEDFY ECOSYSTEM S.L.,[LexDoka is a collaborative platform that turn...,"[Calle Pallars, 108 08018 Barcelona]",...,"[Fintech & Insurtech, \r\n\t\t ...",[Industry 4.0],[Seed (0.25 M EUR - < 1 M EUR)],[2021],[Undisclosed],[Saas],[Business],[None / Not a spinoff],"[[{'amount': '0.3M € (May 2024)', 'date': 'May...",[Funding\n0.3M € (MAY 2024)\nCapital\nInvestor...
3,KOKUAI,KOKUAI S.L.,HealthtechBusiness Services & Software,0.3M €,"Elisabeth Martinez, Oriol Vila, Blast",September 2024,startup/barcelona/kokuai/5838,KOKUAI S.L.,[Kokuai is a Barcelona based start-up founded ...,"[Carrer Llacuna, 162 08018 Barcelona]",...,"[AI & Big Data, \r\n\t\t \t\t\r...",[-],[Seed (0.25 M EUR - < 1 M EUR)],[2020],[From 1 to 5],[Saas],[Business],[None / Not a spinoff],"[[{'amount': '0.3M € (September 2024)', 'date'...",[Funding\n0.3M € (SEPTEMBER 2024)\nCapital\nIn...
4,FLAMAID,FLAMAID S.L.,Hardware,0.3M €,ONA Capital,July 2024,startup/barcelona/flamaid/7137,FLAMAID S.L.,[Flamaid is a safety device equipped with an a...,"[Calle Salvador Espriu, 47 P. 5 Pta. 1 08005 B...",...,[IoT & Sensors],[Social Economy],[Seed (0.25 M EUR - < 1 M EUR)],[2023],[Undisclosed],[Development & Manufacturing],[Consumer],[None / Not a spinoff],"[[{'amount': '0.1M € (November 2024)', 'date':...",[Funding\n0.1M € (NOVEMBER 2024)\nCapital\nInv...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,COCOPOOL,PLACE4PLAN S.L.,E-commerce & MarketplacesTraveltech & Leisure,0.1M €,Fundadors,June 2024,startup/barcelona/cocopool/6433,PLACE4PLAN S.L.,[Platform for renting private pools by the hour.],[Urbanizacion Cabrils Ii (crta Cabrils A Vilas...,...,"[AI & Big Data, \r\n\t\t \t\t\r...",[Sharing Economy],[Pre-seed (< 0.25 M EUR)],[2022],[Undisclosed],[Marketplace],[Consumer],[None / Not a spinoff],"[[{'amount': '0.1M € (June 2024)', 'date': 'Ju...",[Funding\n0.1M € (JUNE 2024)\nCapital\nInvesto...
72,OMASHU,OMASHU S.L.,HealthtechGamingLegaltechSports,0.1M €,"tres efes' (familiars, amics i entusiastes)",April 2024,startup/barcelona/omashu/5895,OMASHU S.L.,[Omashu se especializa en el análisis métrico ...,[Carrer De Pascual I Vila 08028 Barcelona],...,"[AI & Big Data, \r\n\t\t \t\t\r...",[Industry 4.0],[Pre-seed (< 0.25 M EUR)],[2021],[From 6 to 10],"[Consultancy & Agency, \nR+D+I Services, \nSaa...","[Business, \nConsumer]",[None / Not a spinoff],"[[{'amount': '0.1M € (April 2024)', 'date': 'A...",[Funding\n0.1M € (APRIL 2024)\nCapital\nInvest...
73,GET-PIZZA,APPASSIONE S.L.,FoodLogistic Tech,0.0M €,Fundadors,June 2024,startup/barcelona/get-pizza/6718,APPASSIONE S.L.,[Online food ordering platform connecting arti...,[Carrer De La Rosella 17 08035 Barcelona],...,"[E-commerce, \r\n\t\t \t\t\r\n\...",[Smart Cities],[Pre-seed (< 0

In [ ]:
datosFinal.to_csv('datosFinal.csv', index=False)
datosFinal.to_pickle("./datosFinal.pkl") 
